# Introduction to ipydatawidgets

This example assumes you have installed ipydatawidgets (and the appropriate frontend extension) according to the README.

We start with some setup code:

In [1]:
from ipydatawidgets import (
    NDArray, NDArrayWidget, DataUnion, array_serialization, shape_constraints,
    ConstrainedNDArrayWidget
)
from ipywidgets import Widget, widget_serialization
from traitlets import Instance, observe
import numpy as np
np.random.seed(0)

In [2]:
# Let's create some random data to use as an example:
raw_data = 255 * np.random.rand(4, 4, 4)

In [3]:
data_widget = NDArrayWidget(raw_data)

In [4]:
# Create a widget that will use some data
class MyWidget(Widget):
    
    # Add a trait that only accepts raw numpy arrays. Note that we have
    # to add the serializers explicitly:
    array_only = NDArray(np.zeros(0)).tag(sync=True, **array_serialization)
    
    # Add a trait that only accpets a reference to a data widget:
    widget_only = Instance(NDArrayWidget, allow_none=True).tag(
        sync=True, **widget_serialization)
    
    # Add a trait that accepts either an array or a data widget. Note that
    # this sets default serializers for itself. This is the recommended way
    # of adding array traits to widgets.
    data_union = DataUnion(np.zeros(0)).tag(sync=True)


## Simple use:

With a widget and some example data setup, these are the ways it can be used:

In [5]:
# Since we have set valid default values for all traits, we can do an empty init:
w = MyWidget()

First, we assign some valid data to the traits that only accept one type:

In [6]:
w.array_only = raw_data
w.array_only.shape

(4, 4, 4)

In [7]:
w.widget_only = data_widget
w.widget_only.array.shape

(4, 4, 4)

Then, we try assigning some invalid data:

In [8]:
w.widget_only = raw_data

TraitError: The 'widget_only' trait of a MyWidget instance must be a NDArrayWidget or None, but a value of class 'numpy.ndarray' (i.e. array([[[ 139.9474435 ,  182.37328842,  153.7046609 ,  138.94521166],
        [ 108.03197383,  164.70299883,  111.58473887,  227.4021152 ],
        [ 245.73400393,   97.7775873 ,  201.88988471,  134.86820454],
        [ 144.85136308,  236.02714276,   18.11419484,   22.21797142]],

       [[   5.15569135,  212.31806061,  198.42997149,  221.8530978 ],
        [ 249.54767727,  203.78543388,  117.67723737,  199.03493995],
        [  30.1599786 ,  163.17986044,   36.55508829,  240.89057385],
        [ 133.07132205,  105.7387947 ,   67.46168109,  197.42959081]],

       [[ 116.31833472,  144.95065696,    4.79139911,  157.49705175],
        [ 156.08440929,  157.3181692 ,  240.65576002,  173.86417627],
        [  91.67451465,  111.44314822,  177.89595496,   15.35749527],
        [ 170.02551244,  171.01265675,   53.64755307,   32.8762059 ]],

       [[  80.43422

In [9]:
w.array_only = data_widget

TraitError: Object dtype not supported

Note that array-traits will coerce anything numpy can coerce to an array:

In [10]:
w.array_only = list(range(50))
w.array_only

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

Next, we try to assign some data to the union field:

In [11]:
w.data_union = raw_data
w.data_union.shape

(4, 4, 4)

In [12]:
w.data_union = data_widget
w.data_union.array.shape

(4, 4, 4)

## Constraints:

Now, we will add shape and dtype constraints:

In [13]:
color_image_shape_constraint = shape_constraints(None, None, 3)

ColorImageDataWidget = ConstrainedNDArrayWidget(color_image_shape_constraint, dtype=np.uint8)

# Create a widget that can hold color image data, in various forms:
class ColorImageWidget(Widget):
    
    array_only = NDArray(dtype=np.uint8)\
        .tag(sync=True, **array_serialization)\
        .valid(color_image_shape_constraint)
        
    widget_only = Instance(ColorImageDataWidget)\
        .tag(sync=True, **widget_serialization)
    
    data_union = DataUnion(dtype=np.uint8, shape_constraint=color_image_shape_constraint)\
        .tag(sync=True)
    


In [14]:
color_data = raw_data[:, :, :3].astype(np.uint8)
color_data.shape

(4, 4, 3)

In [15]:
color_data_widget = ColorImageDataWidget(array=color_data)

In [16]:
# Initialize with valid data:
wc = ColorImageWidget(
    array_only=color_data,
    widget_only = color_data_widget,
    data_union = color_data_widget
)

Now, try to set various invalid data, that will either fail, or be coerced:

In [17]:
wc.array_only = raw_data   # Fails, since raw_data has wrong size of last axis

TraitError: Dimension 2 is supposed to be size 3, but got dimension 4

In [18]:
wc.array_only = raw_data[:, :, 0]   # Fails, since data has wrong number of dimensions

TraitError: array_only shape expected to have 3 components, but got (4, 4) components

In [19]:
# This will coerce the float data to uint8 (this creates a copy of the data)
wc.array_only = raw_data[:, :, :3]
wc.array_only[:2, :2, :]   # Preview a few values

array([[[139, 182, 153],
        [108, 164, 111]],

       [[  5, 212, 198],
        [249, 203, 117]]], dtype=uint8)

In [20]:
wc.widget_only = data_widget   # Fails, since type of widget is wrong

TraitError: The 'widget_only' trait of a ColorImageWidget instance must be a ConstrainedNDArrayWidget, but a value of class 'ipydatawidgets.ndarray.widgets.NDArrayWidget' (i.e. NDArrayWidget(array=array([[[ 139.9474435 ,  182.37328842,  153.7046609 ,  138.94521166],
        [ 108.03197383,  164.70299883,  111.58473887,  227.4021152 ],
        [ 245.73400393,   97.7775873 ,  201.88988471,  134.86820454],
        [ 144.85136308,  236.02714276,   18.11419484,   22.21797142]],

       [[   5.15569135,  212.31806061,  198.42997149,  221.8530978 ],
        [ 249.54767727,  203.78543388,  117.67723737,  199.03493995],
        [  30.1599786 ,  163.17986044,   36.55508829,  240.89057385],
        [ 133.07132205,  105.7387947 ,   67.46168109,  197.42959081]],

       [[ 116.31833472,  144.95065696,    4.79139911,  157.49705175],
        [ 156.08440929,  157.3181692 ,  240.65576002,  173.86417627],
        [  91.67451465,  111.44314822,  177.89595496,   15.35749527],
        [ 170.02551244,  171.

In [21]:
wc.data_union = data_widget    # Fails, since it cannot coerce data in a widget reference

TraitError: dtypes must match exactly when passing a NDArrayWidget to a dtype constrained DataUnion

In [22]:
data_widget.array = color_data
wc.data_union = data_widget    # Works, even if data_widget itself is unconstrained

data_widget.array = raw_data   # Now not allowed, as our DataUnion trait are constraining data_widget

TraitError: Widget data is constrained by its use in ColorImageWidget(array_only=array([[[139, 182, 153],
        [108, 164, 111],
        [245,  97, 201],
        [144, 236,  18]],

       [[  5, 212, 198],
        [249, 203, 117],
        [ 30, 163,  36],
        [133, 105,  67]],

       [[116, 144,   4],
        [156, 157, 240],
        [ 91, 111, 177],
        [170, 171,  53]],

       [[ 80,  92, 145],
        [252,  26,  53],
        [166,  64, 118],
        [ 40,  28, 167]]], dtype=uint8), data_union=NDArrayWidget(array=array([[[139, 182, 153],
        [108, 164, 111],
        [245,  97, 201],
        [144, 236,  18]],

       [[  5, 212, 198],
        [249, 203, 117],
        [ 30, 163,  36],
        [133, 105,  67]],

       [[116, 144,   4],
        [156, 157, 240],
        [ 91, 111, 177],
        [170, 171,  53]],

       [[ 80,  92, 145],
        [252,  26,  53],
        [166,  64, 118],
        [ 40,  28, 167]]], dtype=uint8)), widget_only=ConstrainedNDArrayWidget(array=